In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import seaborn as sns
import cftime
import src.utils

## set plotting style
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## initialize random number generator
rng = np.random.default_rng()

# Commonly-used operations...
...and examples using ```xarray``` & ```matplotlib```

### opening data

### plotting data

### Regridding data

### grid-wise and grid-to-point correlation

### spatial average

### remove seasonal cycle

### detrending

### interpolation / resampling

# Example: analysis of synthetic climate

#### Simulate synthetic climate

In [ ]:
## specify number of ensemble members and end year for simulation
n_members = 50
tf = 2006

## simulation pre-industrial and warming scenarios
T_PI = src.utils.markov_simulation(ti=850, tf=tf, n_members=n_members, trend=0)
T_warming = src.utils.markov_simulation(ti=1850, tf=tf, n_members=n_members, trend=0.01)

## for convenience, get subset of pre-industrial control which overlaps with warming
T_PI_hist = T_PI.sel(year=T_warming.year)

#### Plot a random sample from each simulation

In [ ]:
# choose a random sample
idx = rng.choice(T_PI.ensemble_member)

# make the plot
fig, ax = plt.subplots(figsize=(4, 3))

ax.plot(
    T_PI_hist.year,
    T_PI_hist.sel(ensemble_member=idx),
    color="black",
    label="P.I. control",
)
ax.plot(
    T_warming.year, T_warming.sel(ensemble_member=idx), color="red", label="warming"
)

## label axes
ax.set_xlabel("Year")
ax.set_ylabel(r"SST anomaly ($^{\circ}C$)")
ax.legend(prop={"size": 10})
ax.set_title("random ensemble member")

plt.show()

#### Plot ensemble mean and spread

In [ ]:
def plot_ensemble_spread(ax, T, color, label=None):
    """plot mean and +/- 1 standard dev. of ensemble on
    given ax object."""

    ## compute stats
    mean = T.mean("ensemble_member")
    std = T.std("ensemble_member")

    ## plot mean
    mean_plot = ax.plot(mean.year, mean, label=label, color=color)

    ## plot spread
    ax.plot(mean.year, mean + std, lw=0.5, c=mean_plot[0].get_color())
    ax.plot(mean.year, mean - std, lw=0.5, c=mean_plot[0].get_color())

    return


## Plot ensemble stats
fig, ax = plt.subplots(figsize=(4, 3))

## plot data
plot_ensemble_spread(ax, T_PI_hist, color="black", label="P.I. control")
plot_ensemble_spread(ax, T_warming, color="red", label="warming")

## label axes
ax.set_xlabel("Year")
ax.set_ylabel(r"SST anomaly ($^{\circ}C$)")
ax.legend(prop={"size": 10})
ax.set_title("Ensemble results")

plt.show()

#### Is the trend significant?
- histogram for PI control (generate by computing trend for 1,000 random 40-year segments)
- compare to trend in warming simulation over last 40 years and for period 1900-1950

# Now it's your turn...
Start by picking a variable/index to analyze (e.g., SST at a specific location or averaged over a specified region), a reanalysis product from the CMIP archive, and a model from the CMIP archive.

#### Model validation
1. Load the data and make a plot.
2. Compute the index for both the reanalysis and the model (in the historical simulation).
3. How do the statistics compare over the overlapping period? (e.g., mean, standard deviation, seasonal cycle, power spectrum).

#### Is there evidence of a climate change signal?
1. Next, compute the index in the model's pre-industrial control simulation (__should this be a last millenium simulation?).
2. Create a histogram of trends in the control simulation by computing the trends of randomly-sampled 40-year segments.
3. Based on this histogram, is the model-simulated trend (from the last 40 years, in the historical simulation) significant?
   
#### What are the projected future changes?
1. Next, compute the index in a future warming scenario (using the same model).
2. How do the statistics/histogram shift in the future warming scenario?